In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import itertools
import sys
import os
import math
sys.path.append('../..')
from joblib import Parallel, delayed
from functools import partial
from hyperopt import hp, tpe, fmin, Trials
from models.optim.sir_dis import SIR
from utils.optim.sir.objective_functions import *
from utils.optim.sir.optimization_methods import *
from matplotlib import colors

# No Intervention

In [ ]:
days = 400
params = {'R0': 2.020218397440326, 'T_treat': 35}
# params=None
grad1, states_base = calculate_opt(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=days, params=params)
height = calculate_opt_height(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=days, params=params)
time = calculate_opt_time(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=days, params=params)
burden = calculate_opt_burden(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=days, capacity=np.array([0.1]), params=params)
print(grad1, height, time, burden)
plt.figure(figsize=(8,8))
plt.plot(ind, states_base[1], label='without intervention')
plt.ylabel('infected fraction')
plt.xlabel('days')
plt.title('Time')
plt.legend()
plt.show()

# Grid Search

### Plot Results for Grid Search

In [ ]:
days = 400
#num_int = 1
min_val, min_params = grid_search(num_int=1, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
ind = np.arange(days)   # the x locations for the groups
width = 0.95        # the width of the bars: can also be len(x) sequence
plt.figure(figsize=(8, 8))
plt.plot(ind, states_base[1], label='without intervention')
plt.plot(ind, states[1], label='1 intervention')

#num_int = 2
min_val, min_params = grid_search(num_int=2, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
plt.plot(ind, states[1], label='2 interventions')

#num_int = 3
min_val, min_params = grid_search(num_int=3, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
plt.plot(ind, states[1], label='3 interventions')

plt.ylabel('infected people')
plt.xlabel('days')
plt.title('Grid_Search: AQALD')
plt.legend()
plt.show()

# Bayesian Search

### Running bayesian search

In [ ]:
days = 400
min_params = tpe_opt(num_int=1, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
ind = np.arange(days)   # the x locations for the groups
width = 0.95        # the width of the bars: can also be len(x) sequence
plt.figure(figsize=(8, 8))
plt.plot(ind, states_base[1], label='without intervention')
plt.plot(ind, states[1], label='1 intervention')

#num_int = 2
min_params = tpe_opt(num_int=2, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
plt.plot(ind, states[1], label='2 interventions')

#num_int = 3
min_params = tpe_opt(num_int=3, days=days, objective='qald')
print(min_params)
val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days)
print(val, height, time, burden)
plt.plot(ind, states[1], label='3 interventions')

plt.ylabel('infected people')
plt.xlabel('days')
plt.title('Bayesian_Search: AQALD')
plt.legend()
plt.show()

### Bayesian search over grid search

In [ ]:
days = 400

def get_stats(min_params, params=None):
    val, states = calculate_opt(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
              intervention_choice=min_params['choice_array'], days=days, params=params)
    height = calculate_opt_height(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
                  intervention_choice=min_params['choice_array'], days=days, params=params)
    time = calculate_opt_time(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
                  intervention_choice=min_params['choice_array'], days=days, params=params)
    burden = calculate_opt_burden(intervention_day=min_params['start_array'], intervention_duration=min_params['duration_array'],\
                  intervention_choice=min_params['choice_array'], days=days, params=params)
    print(val, height, time, burden)
    return(states)

def get_plot(states, fig, num, min_params=None):
    ind = np.arange(len(states[1]))
    ax = fig.add_subplot(1, 1, 1)
    if(not num):
        ax.plot(ind, states[1], label='without intervention', linestyle='dashed')
    else:
        k = len(min_params['start_array'])
        for i in range(k):
            xi = ind[min_params['start_array'][i]:min_params['start_array'][i]+min_params['duration_array'][i]]
            yi = states[1][min_params['start_array'][i]:min_params['start_array'][i]+min_params['duration_array'][i]]
            choice = min_params['choice_array'][i]
            ax.plot(xi, yi, label='strength = '+str(choice)+', '+str(num)+' intervention', linewidth=3)
        ax.plot(ind, states[1], alpha=0.5, label='strength = 0, '+str(num)+' intervention')
    ax.set_ylabel('infected people')
    ax.set_xlabel('days')
    ax.set_title('Grid_Bayesian: AUC-I')
    ax.legend()
    return(fig)

# #num_int=1
params = {'R0': 2.020218397440326, 'T_treat': 35}
# params=None
_, min_params = grid_search(num_int=1, days=days, objective='qald', sir_init=params)
min_params = tpe_grid(num_int=1, days=days, min_params=min_params, objective='qald', iters=500, sir_init=params)
print(min_params)
states = get_stats(min_params, params)

fig = plt.figure(figsize=(8, 8))
fig = get_plot(fig=fig,states=states_base,num=0)
fig = get_plot(fig=fig,states=states,num=1,min_params=min_params)

#num_int = 2
_, min_params = grid_search(num_int=2, days=days, objective='qald', sir_init=params)
min_params = tpe_grid(num_int=2, days=days, min_params=min_params, objective='qald', iters=1000, sir_init=params)
print(min_params)
states = get_stats(min_params, params=params)
fig = get_plot(fig=fig,states=states,num=2,min_params=min_params)

#num_int = 3
_, min_params = grid_search(num_int=3, days=days, objective='qald', sir_init=params)
min_params = tpe_grid(num_int=3, days=days, min_params=min_params, objective='qald', iters=1500, sir_init=params)
print(min_params)
states = get_stats(min_params, params=params)
fig = get_plot(fig=fig,states=states,num=3,min_params=min_params)

# optimal_params = {'choice_array': np.array([0.25, 0.25]), 'start_array': np.array([ 4, 25]), 'duration_array': np.array([ 20, 300])}
# rule_params = {'choice_array': np.array([1.]), 'start_array': np.array([219]), 'duration_array': np.array([80])}
# states = get_stats(optimal_params, params)
# fig = plt.figure(figsize=(8, 8))
# fig = get_plot(fig=fig,states=states_base,num=0)
# fig = get_plot(fig=fig,states=states,num=2,min_params=optimal_params)
# states = get_stats(rule_params, params)
# fig = get_plot(fig=fig,states=states,num='rule',min_params=rule_params)

# Gradient Based Methods - Frank Wolfe Algorithm

In [ ]:
days = 400
best_int_vec = frank_wolfe(days=days, objective='height')
qald, states = run_seir(days=days, int_vec=best_int_vec)
height = seir_height(days=days, int_vec=best_int_vec)
time = seir_time(days=days, int_vec=best_int_vec)
burden = seir_burden(days=days, int_vec=best_int_vec)
print(qald, height, time, burden)
ind = np.arange(days)   # the x locations for the groups
width = 0.95        # the width of the bars: can also be len(x) sequence
plt.figure(figsize=(8, 8))
plt.plot(ind, states_base[1], label='without intervention')
plt.plot(ind, states[1], label='intervention')
plt.ylabel('infected people')
plt.xlabel('days')
plt.legend()
plt.show()

### Discretize FW results

In [ ]:
dis_int_vec = best_int_vec.copy()
dis_int_vec[dis_int_vec<1.25]=1
dis_int_vec[np.logical_and(dis_int_vec<1.75, dis_int_vec>1.25)] = 1.5
dis_int_vec[dis_int_vec>1.75] = 2

In [ ]:
dis_int_vec

In [ ]:
np.sum(dis_int_vec)

In [ ]:
# num=0
# for i in range(len(int_vec)):
#     if(num>=3):
#         break
#     if(int_vec[-i]==1.5):
#         int_vec[-i]=1
#         num+=1
# print(np.sum(int_vec))

In [ ]:
days = 400
qald, states = run_seir(days=days, int_vec=dis_int_vec)
height = seir_height(days=days, int_vec=dis_int_vec)
time = seir_time(days=days, int_vec=dis_int_vec)
burden = seir_burden(days=days, int_vec=dis_int_vec)
print(qald, height, time, burden)
ind = np.arange(days)   # the x locations for the groups
width = 0.95        # the width of the bars: can also be len(x) sequence
plt.figure(figsize=(8, 8))
plt.plot(ind, states_base[1], label='without intervention')
plt.plot(ind, states[1], label='intervention')
plt.ylabel('infected people')
plt.xlabel('days')
plt.legend()
plt.show()